In [23]:
import torch
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from scipy import spatial

In [24]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

In [29]:
s = ['санкт петербург', '', 'москва',
       'волгоградская область | волгоградская обл', 'краснодарский край',
       'новгородская область | новгородская обл', 'алтайский край',
       'воронежская область | воронежская обл', 'московская область',
       'гродненская область', 'ростовская область | ростовская обл',
       'белгородская область | белгородская обл',
       'липецкая область | липецкая обл',
       'свердловская область | свердловская обл',
       'тамбовская область | тамбовская обл',
       'калужская область | калужская обл', 'петропавловск', 'гомель',
       'орловская область | орловская обл',
       'солнечный пгт павлодарская обл',
       'мурманская область | мурманская обл', 'ар крым | крым',
       'архангельская область | архангельская обл',
       'ленинградская область', 'владимирская область | владимирская обл',
       'луганская область', 'брянская область | брянская обл', 'армавир',
       'тульская область | тульская обл',
       'ульяновская область | ульяновская обл', 'приморский край',
       'челябинская область | челябинская обл',
       'курская область | курская обл', 'республика бурятия | бурятия',
       'республика татарстан | татарстан',
       'самарская область | самарская обл',
       'калининградская область | калининградская обл',
       'республика карелия | карелия',
       'ивановская область | ивановская обл',
       'омская область | омская обл',
       'саратовская область | саратовская обл', 'караганда',
       'костромская область | костромская обл', 'республика башкортостан',
       'республика северная осетия алания | северная осетия алания | республика северная осетия алания',
       'кабардино балкарская республика | кабардино балкария',
       'республика ингушетия | ингушетия', 'пермский край',
       'республика дагестан | дагестан', 'республика коми | коми',
       'ямало ненецкий ао | ямало ненецкий округ',
       'вологодская область | вологодская обл',
       'псковская область | псковская обл',
       'удмуртская республика | удмуртия',
       'оренбургская область | оренбургская обл',
       'смоленская область | смоленская обл', 'хабаровский край',
       'бишкек', 'тюменская область | тюменская обл', 'ош',
       'брестская область', 'тверская область | тверская обл',
       'новосибирская область | новосибирская обл', 'нижний новгород',
       'кемеровская область | кемеровская обл',
       'ярославская область | ярославская обл',
       'ханты мансийский ао югра | ханты мансийский округ югра',
       'рязанская область | рязанская обл',
       'кировская область | кировская обл',
       'республика марий эл | марий эл',
       'нижегородская область | нижегородская обл',
       'астраханская область | астраханская обл', 'красноярский край',
       'чеченская республика | чечня', 'республика калмыкия | калмыкия',
       'республика мордовия | мордовия', 'ставропольский край',
       'пензенская область | пензенская обл',
       'чувашская республика | чувашия', 'республика хакасия | хакасия',
       'иркутская область | иркутская обл',
       'карачаево черкесская республика | карачаево черкессия',
       'курганская область | курганская обл', 'донецкая область',
       'республика саха якутия | якутия | республика саха', 'витебск',
       'республика адыгея | адыгея', 'томская область | томская обл',
       'гомельская область', 'могилев', 'бухара', 'витебская область',
       'киев | київ', 'шымкент']
s1 = 'московская область'#s[0]
s = s[1:]
S = []
S1 = embed_bert_cls(s1, model, tokenizer)
for name in s:
    S.append(1-spatial.distance.cosine(S1,embed_bert_cls(name, model, tokenizer)))
res = pd.DataFrame(data={'name':s,'dist':S})
res = res.sort_values(by=['dist'],ascending=False).reset_index(drop=True)

In [30]:
s1

'московская область'

In [32]:
res.head(30)

,name,dist
0,московская область,1.000000
1,ленинградская область,0.963156
2,брестская область,0.952512
3,гродненская область,0.951504
4,гомельская область,0.943645
5,томская область | томская обл,0.939829
6,луганская область,0.925242
7,донецкая область,0.921614
8,пермский край,0.915727
9,республика башкортостан,0.909795


In [ ]:
res

,name,dist
0,обязанности управление автопогрузчиком требова...,0.922842
1,требуется водитель на автомобиле ресторана тре...,0.916788
2,обязанности управление служебным автомобилем к...,0.906334
3,обязанности доставка груза заказчику приемка э...,0.904785
4,ресторан приглашает на работу водителя доставк...,0.901963
...,...,...
209,персональный водитель,0.306974
210,водитель для партнеров,0.306224
211,в круглосуточный ресторан,0.301646
212,nbsp нов ше,0.249049
